# データサイエンス100本ノック（構造化データ加工編） - Python

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

# 0. ver1.2目的
> ビジネス的に有益な目的変数（2値）の探索 \
> ロジスティックモデルの構築 \
> ※できれば初心者が構築してもAR0.7くらい出るのが望ましい

## 0-1. シチュエーション設定
> 顧客：〇〇ストアー \
> 依頼内容：店舗の決済データと商品データおよび顧客情報から有益な知見が欲しい（超アバウト）\

## 0-2. 目的変数設定
> 1．将来の売上金額（全店？加盟店ごと？個人？） \
> 2．各個人に対する将来の利用推定 \
>   2-1. 予測期間に1回でも、観測期間の購入金額月平均

# 1. データの確認

## 1-1. 顧客情報

In [4]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


## 1-2. 商品カテゴリ情報

In [5]:
df_category.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,04,惣菜,0401,御飯類,040101,弁当類
1,04,惣菜,0401,御飯類,040102,寿司類
2,04,惣菜,0402,佃煮類,040201,魚介佃煮類
3,04,惣菜,0402,佃煮類,040202,海草佃煮類
4,04,惣菜,0402,佃煮類,040203,野菜佃煮類


## 1-3. 商品情報

In [6]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,04,0401,040101,198.0,149.0
1,P040101002,04,0401,040101,218.0,164.0
2,P040101003,04,0401,040101,230.0,173.0
3,P040101004,04,0401,040101,248.0,186.0
4,P040101005,04,0401,040101,268.0,201.0


## 1-4. 決済情報

In [7]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


## 1-5. 店舗情報

In [8]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


## 1-6. 地理情報

In [9]:
df_geocode.head()

,postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
0,060-0000,北海道,札幌市中央区,None,None,None,北海道札幌市中央区,141.34103,43.05513
1,064-0941,北海道,札幌市中央区,旭ケ丘,None,None,北海道札幌市中央区旭ケ丘,141.31972,43.04223
2,060-0042,北海道,札幌市中央区,大通西,None,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102
3,060-0042,北海道,札幌市中央区,大通西,None,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080
4,060-0042,北海道,札幌市中央区,大通西,None,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086
